# CS2007 - ARTIFICIAL INTELLIGENCE

## CIA-1

**Submitted by:**

**Name:** Raahul R <br>
**Dept:** B.Tech CSE (Cyber Security), 3rd Year <br>
**RegNo:** 22011103043 <br>
**GitHub:** raahulcodez

**Implement the following AI search algorithms:**

- British Museum Search
- Depth First Search 
- Breadth First Search 
- Hill Climbing
- Beam Search
- Oracle
- Branch and Bound
- Branch and Bound + Extended List (deadhorse)
- Branch and Bound + Cost + Heuristics
- A*

### Hard coding a sample graph

In [13]:
graph = {
    'A': {'heuristic': 7, 'edges': {'B': 1, 'C': 4}},
    'B': {'heuristic': 6, 'edges': {'D': 2, 'E': 5}},
    'C': {'heuristic': 2, 'edges': {'D': 2, 'F': 5}},
    'D': {'heuristic': 1, 'edges': {'E': 1, 'G': 3}},
    'E': {'heuristic': 4, 'edges': {'H': 2}},
    'F': {'heuristic': 1, 'edges': {'H': 3}},
    'G': {'heuristic': 3, 'edges': {'H': 1}},
    'H': {'heuristic': 0, 'edges': {}}
}

In [14]:
def calculate_path_cost(graph, path):
    total_cost = 0
    for i in range(len(path) - 1):
        current_node = path[i]
        next_node = path[i + 1]
        # Add the cost of the edge between current_node and next_node
        total_cost += graph[current_node]['edges'][next_node]
    return total_cost

**British Museum Search (BMS):**

In [15]:
def british_museum_search(graph, start, goal):
    # Initialize the frontier and explored set
    frontier = [start]  # Start with the start node
    explored = set()
    paths = {start: None}  # To store paths

    while frontier:
        # Sort the frontier to explore nodes in alphabetical order
        frontier.sort()
        current_node = frontier.pop(0)  # Get the first node from the frontier

        # Check if we reached the goal
        if current_node == goal:
            return reconstruct_path(paths, start, goal)

        explored.add(current_node)

        # Explore neighbors in alphabetical order
        for neighbor in sorted(graph[current_node]['edges']):
            if neighbor not in explored and neighbor not in frontier:
                frontier.append(neighbor)
                paths[neighbor] = current_node  # Update the path

    return None  # Return None if no path is found

def reconstruct_path(paths, start, goal):
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = paths[current]
    path.reverse()
    return path

path = british_museum_search(graph, "A", "H")
print(f"Path from A to H: {path}")
print(f"Path cost: {calculate_path_cost(graph, path)}")

Path from A to H: ['A', 'B', 'E', 'H']
Path cost: 8


**Oracle:**

**Depth First Search (DFS):**

In [16]:
def depth_first_search(graph, start, goal, path=None, visited=None):
    if path is None:
        path = []
    if visited is None:
        visited = set()

    path.append(start)
    visited.add(start)

    # If we've reached the goal node, return the current path
    if start == goal:
        return path

    # Recursively explore each neighbor of the current node
    for neighbor in graph[start]['edges']:
        if neighbor not in visited:
            result = depth_first_search(graph, neighbor, goal, path, visited)
            if result:  # If a valid path is found
                return result

    # Backtrack if no valid path is found from this node
    path.pop()
    return None

path = depth_first_search(graph, "A", "H")

print(f"Path from A to H: {path}")
print(f"Path cost: {calculate_path_cost(graph, path)}")

Path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6


**Breadth first search (BFS):**

In [22]:
from collections import deque

def breadth_first_search(graph, start, goal):
    queue = deque([[start]])  # Queue to store paths
    visited = set()           # Set to store visited nodes

    while queue:
        path = queue.popleft()  
        node = path[-1]         

        if node == goal:
            return path

        # If node has not been visited, mark it as visited
        if node not in visited:
            visited.add(node)

            # Explore the neighbors of the current node
            for neighbor in graph[node]['edges']:
                new_path = list(path) 
                new_path.append(neighbor)
                queue.append(new_path)  

    return None 

path = breadth_first_search(graph, "A", "H")

print(f"Path from A to H: {path}")
print(f"Path cost: {calculate_path_cost(graph, path)}")

Path from A to H: ['A', 'B', 'E', 'H']
Path cost: 8


**Hill Climbing:**

In [23]:
def hill_climbing(graph, start, goal):
    path = [start]
    current_node = start

    while current_node != goal:
        neighbors = graph[current_node]['edges']
        
        # If there are no neighbors to move to, return None (stuck at local maxima)
        if not neighbors:
            return None

        # Choose the neighbor with the lowest heuristic value
        next_node = min(neighbors, key=lambda neighbor: graph[neighbor]['heuristic'])

        # If the next node's heuristic is not better, stop (local maxima)
        if graph[next_node]['heuristic'] >= graph[current_node]['heuristic']:
            return None

        # Move to the next node and add it to the path
        path.append(next_node)
        current_node = next_node

    return path

# Perform hill climbing search
path = hill_climbing(graph, "A", "H")

# Print the path and its cost if found
if path:
    print(f"Path from A to H: {path}")
    print(f"Path cost: {calculate_path_cost(graph, path)}")
else:
    print("No path found (stuck at local maxima)")

No path found (stuck at local maxima)


**Beam Search (width - 2):**

In [24]:
import heapq

def beam_search(graph, start, goal, beam_width):
    # Priority queue for nodes to explore, initialized with the start node and a cost of 0
    priority_queue = [(0, [start])]  # (current cost, path)

    while priority_queue:
        # Only keep the top 'beam_width' number of paths
        next_queue = []

        # Process all current paths in the priority queue
        for current_cost, path in priority_queue:
            current_node = path[-1]

            # If we reach the goal, return the path and its cost
            if current_node == goal:
                return path, current_cost

            # Explore neighbors of the current node
            for neighbor, edge_cost in graph[current_node]['edges'].items():
                if neighbor not in path:  # Avoid cycles
                    new_cost = current_cost + edge_cost
                    new_path = path + [neighbor]
                    next_queue.append((new_cost, new_path))

        # Sort next_queue by cost and trim it to the beam width
        next_queue = sorted(next_queue, key=lambda x: x[0])[:beam_width]

        # Update priority queue with the top beam_width paths
        priority_queue = next_queue

    # If no path found to the goal
    return None, float('inf')

# Perform Beam Search with beam width 2
beam_width = 2
path, cost = beam_search(graph, "A", "H", beam_width)

# Print the best path and its cost if found
if path:
    print(f"Path from A to H: {path}")
    print(f"Path cost: {cost}")
else:
    print("No path found")


Path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6


**Oracle:**

In [25]:
def oracle_search(graph, start, goal, OR):
    path = [start]
    current_node = start
    current_cost = 0

    while current_node != goal:
        # Get the neighbors of the current node
        neighbors = list(graph[current_node]['edges'].keys())

        if not neighbors:  # If no neighbors, path is stuck
            return None, float('inf')

        # Find the neighbor with the smallest edge cost
        next_node = min(neighbors, key=lambda node: graph[current_node]['edges'][node])
        edge_cost = graph[current_node]['edges'][next_node]

        # Add the cost of moving to the next node
        current_cost += edge_cost

        # If total cost exceeds the Oracle Rate (OR), discard this path
        if current_cost > OR:
            return None, float('inf')

        # Move to the next node and add to the path
        path.append(next_node)
        current_node = next_node

        # If we are stuck in a cycle, terminate
        if len(path) > len(graph):  # This prevents infinite loops
            return None, float('inf')

    # Calculate the cost of the final path using the provided function
    path_cost = calculate_path_cost(graph, path)
    return path, path_cost

# Perform Oracle Search with Oracle Rate 6
path, cost = oracle_search(graph, "A", "H", OR=6)
if path:
    print(f"Path from A to H: {path}")
    print(f"Path cost: {cost}")
else:
    print("No path found")


Path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6


**Branch and Bound:**

In [20]:
import heapq

def branch_and_bound(graph, start, goal):
    # Priority queue for nodes to explore, initialized with the start node and a cost of 0
    priority_queue = [(0, [start])]  # (current cost, path)
    best_path = None
    best_cost = float('inf')  # Initially, set the best cost to infinity

    while priority_queue:
        current_cost, path = heapq.heappop(priority_queue)
        current_node = path[-1]

        # If we reach the goal, check if it's the best path so far
        if current_node == goal:
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = path
            continue

        # Explore neighbors of the current node
        for neighbor, edge_cost in graph[current_node]['edges'].items():
            if neighbor not in path:  # Avoid cycles
                new_cost = current_cost + edge_cost
                new_path = path + [neighbor]

                # Only explore if this new path's cost is better than the best path's cost so far
                if new_cost < best_cost:
                    heapq.heappush(priority_queue, (new_cost, new_path))

    return best_path, best_cost

# Perform Branch and Bound search
path, cost = branch_and_bound(graph, "A", "H")

# Print the best path and its cost if found
if path:
    print(f"Best path from A to H: {path}")
    print(f"Path cost: {cost}")
else:
    print("No path found")

Best path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6


**Branch and Bound + Extended List (Deadhorse):**

In [26]:
import heapq

def branch_and_bound_deadhorse(graph, start, goal):
    # Priority queue for Branch and Bound (heapq) stores (cost, path)
    priority_queue = [(0, [start])]
    best_cost = float('inf')
    best_path = None
    deadhorse = set()  # Extended list to avoid suboptimal paths

    while priority_queue:
        current_cost, path = heapq.heappop(priority_queue)
        current_node = path[-1]

        # If current path exceeds the best known cost, skip it (pruning)
        if current_cost >= best_cost:
            continue

        # If the current node is the goal and cost is better than best_cost
        if current_node == goal:
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = path
            continue

        # Add current path to the Deadhorse list to avoid revisiting
        deadhorse.add(tuple(path))

        # Explore neighbors of the current node
        for neighbor, edge_cost in graph[current_node]['edges'].items():
            if neighbor not in path:
                new_path = path + [neighbor]
                new_cost = current_cost + edge_cost
                
                # Skip if the new path is already in Deadhorse list
                if tuple(new_path) not in deadhorse:
                    heapq.heappush(priority_queue, (new_cost, new_path))

    return best_path, best_cost

# Perform Branch and Bound + Deadhorse Search
path, cost = branch_and_bound_deadhorse(graph, "A", "H")

# Print the resulting path and cost if found
if path:
    print(f"Branch and Bound + Deadhorse path from A to H: {path}")
    print(f"Path cost: {cost}")
else:
    print("No valid path found.")


Branch and Bound + Deadhorse path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6


**Branch and Bound + Cost + Heuristic:**

In [27]:
import heapq

def branch_and_bound_with_heuristics(graph, start, goal):
    # Priority queue stores tuples (total_cost, path_cost, node, path)
    priority_queue = [(graph[start]['heuristic'], 0, start, [start])]
    best_cost = float('inf')
    best_path = None
    visited = set()

    while priority_queue:
        total_cost, current_cost, current_node, path = heapq.heappop(priority_queue)

        # If we reached the goal node, check if this path is better
        if current_node == goal:
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = path
            continue
        
        # Mark the current node as visited
        visited.add(current_node)

        # Explore neighbors
        for neighbor, edge_cost in graph[current_node]['edges'].items():
            if neighbor not in visited:
                new_path = path + [neighbor]
                new_cost = current_cost + edge_cost
                heuristic_cost = graph[neighbor]['heuristic']
                total_new_cost = new_cost + heuristic_cost

                # Only add to queue if total_new_cost is better than the best known cost
                if total_new_cost < best_cost:
                    heapq.heappush(priority_queue, (total_new_cost, new_cost, neighbor, new_path))

    return best_path, best_cost

# Perform Branch and Bound with cost + heuristics
path, cost = branch_and_bound_with_heuristics(graph, "A", "H")

# Print the resulting path and cost if found
if path:
    print(f"Branch and Bound (Cost + Heuristics) path from A to H: {path}")
    print(f"Path cost: {cost}")
else:
    print("No valid path found.")

Branch and Bound (Cost + Heuristics) path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6


**A***

In [28]:
import heapq

def a_star(graph, start, goal):
    # Priority queue stores tuples (f_cost, g_cost, node, path)
    priority_queue = [(graph[start]['heuristic'], 0, start, [start])]
    visited = set()

    while priority_queue:
        # Pop the node with the lowest f(n) = g(n) + h(n)
        f_cost, g_cost, current_node, path = heapq.heappop(priority_queue)

        # If we reached the goal node, return the path and cost
        if current_node == goal:
            return path, g_cost

        # Mark the current node as visited
        if current_node in visited:
            continue
        visited.add(current_node)

        # Explore neighbors
        for neighbor, edge_cost in graph[current_node]['edges'].items():
            if neighbor not in visited:
                new_g_cost = g_cost + edge_cost  # Update the path cost (g(n))
                heuristic_cost = graph[neighbor]['heuristic']  # Heuristic cost (h(n))
                new_f_cost = new_g_cost + heuristic_cost  # f(n) = g(n) + h(n)
                new_path = path + [neighbor]
                
                # Add to priority queue
                heapq.heappush(priority_queue, (new_f_cost, new_g_cost, neighbor, new_path))

    return None, float('inf')  # Return None if no path is found

# Perform A* Search
path, cost = a_star(graph, "A", "H")

# Print the resulting path and cost if found
if path:
    print(f"A* path from A to H: {path}")
    print(f"Path cost: {cost}")
else:
    print("No valid path found.")


A* path from A to H: ['A', 'B', 'D', 'E', 'H']
Path cost: 6
